In [1]:
from google.colab import drive
drive.mount("/content/drive")
DRIVE_SRC = "/content/drive/MyDrive/IIT-Projects/third-umpire-decision-automation/backend/experiments/src"
!rm -r -f ./sample_data
!cp -r /content/drive/MyDrive/IIT-Projects/third-umpire-decision-automation/.creds/.aws ~/
!cp -r $DRIVE_SRC ./src

Mounted at /content/drive


In [2]:
! pip install -qq sagemaker ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.6/718.6 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 17.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 630.1/630.1 kB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 99.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.0/527.0 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 kB 23.0 MB/s eta 0:00:00


In [3]:
from sagemaker.s3 import S3Downloader
S3Downloader.download("s3://third-umpire-decision-automation-osura/datasets/object-dataset-1", "./datasets/object-dataset-1")
!mv ./datasets/object-dataset-1/cricket.yaml ./

# Detect

In [4]:
from ultralytics import YOLO

In [5]:
model = YOLO('yolov8n.pt')

100%|██████████| 6.23M/6.23M [00:00<00:00, 68.9MB/s]


In [6]:
model.train(data='./cricket.yaml', epochs=50)

Ultralytics YOLOv8.0.81 🚀 Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=./cricket.yaml, epochs=50, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, simplify=Fals

In [ ]:
res = model.predict("/home/osurasenith/Projects/third-umpire-decision-automation/backend/experiments/data/Cricket-Semantic-Segmentation/images/2022-08-24 (5).png")


image 1/1 /home/osurasenith/Projects/third-umpire-decision-automation/backend/experiments/data/Cricket-Semantic-Segmentation/images/2022-08-24 (5).png: 384x640 7 persons, 41.2ms
Speed: 0.4ms preprocess, 41.2ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)


In [ ]:
res[0].boxes

WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.


ultralytics.yolo.engine.results.Boxes object with attributes:

boxes: tensor([[9.2935e+02, 4.7160e+02, 9.9675e+02, 6.5322e+02, 8.0926e-01, 0.0000e+00],
        [1.0140e+03, 4.9032e+02, 1.0972e+03, 6.8439e+02, 7.9457e-01, 0.0000e+00],
        [7.7993e+02, 4.0239e+02, 8.3196e+02, 5.8085e+02, 6.7104e-01, 0.0000e+00],
        [1.2101e+03, 1.3299e+02, 1.2815e+03, 2.1766e+02, 6.4718e-01, 0.0000e+00],
        [1.1008e+03, 1.1816e+02, 1.1597e+03, 2.0196e+02, 6.2679e-01, 0.0000e+00],
        [9.1617e+02, 2.2891e+02, 9.6383e+02, 3.6585e+02, 5.6057e-01, 0.0000e+00],
        [9.7013e+02, 1.2499e+02, 1.0056e+03, 2.1580e+02, 3.0256e-01, 0.0000e+00]])
cls: tensor([0., 0., 0., 0., 0., 0., 0.])
conf: tensor([0.8093, 0.7946, 0.6710, 0.6472, 0.6268, 0.5606, 0.3026])
data: tensor([[9.2935e+02, 4.7160e+02, 9.9675e+02, 6.5322e+02, 8.0926e-01, 0.0000e+00],
        [1.0140e+03, 4.9032e+02, 1.0972e+03, 6.8439e+02, 7.9457e-01, 0.0000e+00],
        [7.7993e+02, 4.0239e+02, 8.3196e+02, 5.8085e+02, 6.7104e-01, 0.0

In [ ]:
res[0]

ultralytics.yolo.engine.results.Results object with attributes:

boxes: ultralytics.yolo.engine.results.Boxes object
keypoints: None
keys: ['boxes']
masks: None
names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'c

# Segment

In [ ]:
from ultralytics import YOLO
import cv2 as cv
import matplotlib.pyplot as plt
from src.data.visualize import drawSegments

In [ ]:
model = YOLO('yolov8n-seg.pt')
model.train(data='coco128-seg.yaml', epochs=1, imgsz=640)

In [ ]:
img_path = "/content/datasets/coco128-seg/images/train2017/000000000009.jpg"
lbl_path = "/content/datasets/coco128-seg/labels/train2017/000000000009.txt"

In [ ]:
img = cv.imread(img_path)
plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
plt.show()

In [ ]:
pts_txt = ""
with open(lbl_path) as handler:
    pts_txt = handler.read()

segment_drawn_img = drawSegments(img, pts_txt, color_overlay_ratio=0.5)
plt.imshow(cv.cvtColor(segment_drawn_img, cv.COLOR_BGR2RGB))
plt.show()